# Data Preparation for GA blockgroup and precinct data

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import maup

## Import shapefiles with decennial data

In [5]:
ga_bgs_decennial = gpd.read_file("../shapes/Georgia/GA_block_groups/GA_block_groups.shp")

In [6]:
ga_blocks = gpd.read_file("../shapes/Georgia/GA_blocks_10/GA_blocks_2010.shp")

## Clean ACS data from NHGIS

* TOTPOP broked down by race and hispanic origin
* aggregate VAP counts

In [7]:
ga_bgs_acs = pd.read_csv("data/nhgis0028_csv/nhgis0028_ds239_20185_2018_blck_grp.csv")

In [8]:
ga_bgs_acs["GEOID"] = ga_bgs_acs["GISJOIN"].apply(lambda geoid: geoid[1:3] + geoid[4:7] + geoid[8:])

In [9]:
pop_cols_to_rename = {"AJWVE001": "TOTPOP18",
                     "AJWVE003": "NH_WHITE18",
                     "AJWVE004": "NH_BLACK18",
                     "AJWVE005": "NH_AMIN18",
                     "AJWVE006": "NH_ASIAN18",
                     "AJWVE007": "NH_NHPI18",
                     "AJWVE008": "NH_OTHER18",
                     "AJWVE009": "NH_2MORE18",
                     "AJWVE012": "HISP18",
                     "AJWVE013": "H_WHITE18",
                     "AJWVE014": "H_BLACK18",
                     "AJWVE015": "H_AMIN18",
                     "AJWVE016": "H_ASIAN18",
                     "AJWVE017": "H_NHPI18",
                     "AJWVE018": "H_OTHER18",
                     "AJWVE019": "H_2MORE18"}

In [10]:
ga_bgs_acs.rename(columns=pop_cols_to_rename,inplace=True)

In [11]:
u18_cols = (["AJWBE{:03}".format(i) for i in range(3,7)] 
          + ["AJWBE{:03}".format(i) for i in range(27,31)])
vap_cols = (["AJWBE{:03}".format(i) for i in range(7,26)] 
          + ["AJWBE{:03}".format(i) for i in range(31,50)])

In [12]:
## Sanity check that U18 + VAP is equal to TOTPOP for all blocks
(ga_bgs_acs[u18_cols].sum(axis=1) + ga_bgs_acs[vap_cols].sum(axis=1) == ga_bgs_acs.TOTPOP18).all()

True

In [13]:
ga_bgs_acs["U18_18"] = ga_bgs_acs[u18_cols].sum(axis=1)
ga_bgs_acs["VAP18"] = ga_bgs_acs[vap_cols].sum(axis=1)

In [14]:
ga_bgs_acs.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,AJWVM015,AJWVM016,AJWVM017,AJWVM018,AJWVM019,AJWVM020,AJWVM021,GEOID,U18_18,VAP18
0,G13000109501001,2014-2018,NaN,NaN,Georgia,13,Appling County,1,NaN,NaN,...,13,13,13,13,13,13,13,130019501001,118,769
1,G13000109501002,2014-2018,NaN,NaN,Georgia,13,Appling County,1,NaN,NaN,...,13,13,13,13,13,13,13,130019501002,583,1541
2,G13000109502001,2014-2018,NaN,NaN,Georgia,13,Appling County,1,NaN,NaN,...,13,13,13,200,13,13,13,130019502001,349,1362
3,G13000109502002,2014-2018,NaN,NaN,Georgia,13,Appling County,1,NaN,NaN,...,13,13,13,13,13,13,13,130019502002,267,1028
4,G13000109502003,2014-2018,NaN,NaN,Georgia,13,Appling County,1,NaN,NaN,...,13,13,13,13,13,13,13,130019502003,244,1051


In [15]:
cols_to_drop = ['COUSUBA','PLACEA','CONCITA', 'RES_ONLYA','TRUSTA','AIANHHA','AITSCEA',
                'ANRCA','CBSAA','METDIVA','CSAA','NECTAA','NECTADIVA','CNECTAA', 'UAA',
                'SLDUA','SLDLA','ZCTA5A','SUBMCDA','SDELMA','SDSECA','SDUNIA',]

In [16]:
cols_to_drop.extend(list(ga_bgs_acs.filter(regex='AJW')))

In [17]:
ga_bgs_acs_clean = ga_bgs_acs.drop(columns=cols_to_drop)

In [18]:
ga_bgs_acs_clean.to_csv("data/GA_ACS_VAP_POP_bgs_2014-18.csv")

In [19]:
ga_bgs_acs_clean.columns

Index(['GISJOIN', 'YEAR', 'REGIONA', 'DIVISIONA', 'STATE', 'STATEA', 'COUNTY',
       'COUNTYA', 'TRACTA', 'BLKGRPA', 'CDCURRA', 'PUMA5A', 'BTTRA', 'BTBGA',
       'NAME_E', 'TOTPOP18', 'NH_WHITE18', 'NH_BLACK18', 'NH_AMIN18',
       'NH_ASIAN18', 'NH_NHPI18', 'NH_OTHER18', 'NH_2MORE18', 'HISP18',
       'H_WHITE18', 'H_BLACK18', 'H_AMIN18', 'H_ASIAN18', 'H_NHPI18',
       'H_OTHER18', 'H_2MORE18', 'NAME_M', 'GEOID', 'U18_18', 'VAP18'],
      dtype='object')

## Merge ACS with decennial shapefile

In [20]:
ga_bgs_decennial.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'group',
       'ALAND10', 'AWATER10', 'TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN',
       'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE',
       'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP',
       'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP',
       '2MOREVAP', 'geometry'],
      dtype='object')

In [21]:
cols = ['TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN','NH_ASIAN', 'NH_NHPI', 
        'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE', 'H_BLACK', 'H_AMIN', 
        'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 
        'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP','2MOREVAP']

In [22]:
ga_bgs_decennial.rename(columns={k: "{}10".format(k) for k in cols}, inplace=True)

In [23]:
ga_bgs = pd.merge(ga_bgs_decennial, ga_bgs_acs_clean, on="GEOID")

In [24]:
ga_bgs.to_file("data/GA_blockgroups_2018/GA_blockgroups_2018.shp")